# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-30 02:32:18] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=38132, mem_fraction_static=0.849, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=291044774, constrained_json_whitespace_pattern=None, watchdog_timeout=3

[2025-06-30 02:32:18] Inferred chat template from model path: llama-2


[2025-06-30 02:32:29] Attention backend not set. Use flashinfer backend by default.
[2025-06-30 02:32:29] Init torch distributed begin.


[2025-06-30 02:32:31] Init torch distributed ends. mem usage=0.00 GB


[2025-06-30 02:32:33] Load weight begin. avail mem=61.98 GB


...

[2025-06-30 02:32:37] Init torch distributed begin.
[2025-06-30 02:32:37] Init torch distributed ends. mem usage=0.00 GB
[2025-06-30 02:32:37] Load weight begin. avail mem=55.16 GB
[2025-06-30 02:32:37] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.36s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.36s/it]

[2025-06-30 02:32:39] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=54.23 GB, mem usage=0.93 GB.
[2025-06-30 02:32:39] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-06-30 02:32:39] Memory pool end. avail mem=53.92 GB


[2025-06-30 02:32:40] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=54.14 GB
[2025-06-30 02:32:40] INFO:     Started server process [1981963]
[2025-06-30 02:32:40] INFO:     Waiting for application startup.
[2025-06-30 02:32:40] INFO:     Application startup complete.
[2025-06-30 02:32:40] INFO:     Uvicorn running on http://127.0.0.1:38132 (Press CTRL+C to quit)


[2025-06-30 02:32:40] INFO:     127.0.0.1:45950 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-30 02:32:41] INFO:     127.0.0.1:45960 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-30 02:32:41] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-30 02:33:44] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, #token: 7, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-06-30 02:34:13] INFO:     127.0.0.1:45972 - "POST /generate HTTP/1.1" 200 OK
[2025-06-30 02:34:13] The server is fired up and ready to roll!


[2025-06-30 02:34:17] INFO:     127.0.0.1:40240 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-06-30 02:34:17] Child process unexpectedly failed with exitcode=9. pid=1983036


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-30 02:34:28] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=33368, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=37716493, constrained_json_whitespace_pattern=None, watchdog_timeout=300,

[2025-06-30 02:34:28] Inferred chat template from model path: llama-2


[2025-06-30 02:34:40] Attention backend not set. Use flashinfer backend by default.
[2025-06-30 02:34:40] Init torch distributed begin.


[2025-06-30 02:34:41] Init torch distributed ends. mem usage=0.00 GB


[2025-06-30 02:34:42] Load weight begin. avail mem=78.50 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.22s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.22s/it]

[2025-06-30 02:34:48] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=53.72 GB, mem usage=1.44 GB.
[2025-06-30 02:34:48] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-06-30 02:34:48] Memory pool end. avail mem=53.41 GB


[2025-06-30 02:34:48] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=53.59 GB
[2025-06-30 02:34:48] INFO:     Started server process [1989515]
[2025-06-30 02:34:48] INFO:     Waiting for application startup.
[2025-06-30 02:34:48] INFO:     Application startup complete.
[2025-06-30 02:34:48] INFO:     Uvicorn running on http://127.0.0.1:33368 (Press CTRL+C to quit)


[2025-06-30 02:34:48] INFO:     127.0.0.1:36374 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-30 02:34:49] INFO:     127.0.0.1:36388 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-30 02:34:49] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-30 02:34:52] INFO:     127.0.0.1:36404 - "POST /generate HTTP/1.1" 200 OK
[2025-06-30 02:34:52] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-30 02:34:53] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-30 02:34:54] INFO:     127.0.0.1:39616 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-06-30 02:34:54] Child process unexpectedly failed with exitcode=9. pid=1989926
[2025-06-30 02:34:54] Child process unexpectedly failed with exitcode=9. pid=1989680


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-30 02:35:05] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=34640, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=748652021, constrained_json_whitespace_pattern=None,

[2025-06-30 02:35:18] Casting torch.bfloat16 to torch.float16.


[2025-06-30 02:35:18] Casting torch.bfloat16 to torch.float16.
[2025-06-30 02:35:18] Attention backend not set. Use flashinfer backend by default.
[2025-06-30 02:35:18] Init torch distributed begin.


[2025-06-30 02:35:19] Init torch distributed ends. mem usage=0.00 GB


[2025-06-30 02:35:20] Load weight begin. avail mem=62.09 GB


...

[2025-06-30 02:35:36] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=59.15 GB


[2025-06-30 02:35:36] INFO:     Started server process [1991952]
[2025-06-30 02:35:36] INFO:     Waiting for application startup.
[2025-06-30 02:35:36] INFO:     Application startup complete.
[2025-06-30 02:35:36] INFO:     Uvicorn running on http://127.0.0.1:34640 (Press CTRL+C to quit)


[2025-06-30 02:35:37] INFO:     127.0.0.1:46764 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-30 02:35:37] INFO:     127.0.0.1:46774 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-30 02:35:37] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-30 02:35:40] INFO:     127.0.0.1:46788 - "POST /generate HTTP/1.1" 200 OK
[2025-06-30 02:35:40] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-30 02:35:42] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-30 02:35:42] INFO:     127.0.0.1:46800 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-06-30 02:35:42] Child process unexpectedly failed with exitcode=9. pid=1992590


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-30 02:35:53] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=36727, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=376774444, constrained_json_whitespace_pattern=None, watchdog

[2025-06-30 02:36:04] Casting torch.bfloat16 to torch.float16.


[2025-06-30 02:36:04] Casting torch.bfloat16 to torch.float16.
[2025-06-30 02:36:04] Attention backend not set. Use flashinfer backend by default.
[2025-06-30 02:36:04] Init torch distributed begin.


[2025-06-30 02:36:04] Init torch distributed ends. mem usage=0.00 GB


[2025-06-30 02:36:06] Load weight begin. avail mem=78.50 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.02it/s]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.02it/s]

[2025-06-30 02:36:20] Load weight end. type=LlamaForCausalLMEagle3, dtype=torch.float16, avail mem=58.29 GB, mem usage=1.82 GB.
[2025-06-30 02:36:20] KV Cache is allocated. #tokens: 20480, K size: 0.04 GB, V size: 0.04 GB
[2025-06-30 02:36:20] Memory pool end. avail mem=58.21 GB


[2025-06-30 02:36:21] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=58.89 GB


[2025-06-30 02:36:21] INFO:     Started server process [1994471]
[2025-06-30 02:36:21] INFO:     Waiting for application startup.
[2025-06-30 02:36:21] INFO:     Application startup complete.
[2025-06-30 02:36:21] INFO:     Uvicorn running on http://127.0.0.1:36727 (Press CTRL+C to quit)
[2025-06-30 02:36:21] INFO:     127.0.0.1:56994 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-30 02:36:22] INFO:     127.0.0.1:57002 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-30 02:36:23] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-30 02:36:25] INFO:     127.0.0.1:57014 - "POST /generate HTTP/1.1" 200 OK
[2025-06-30 02:36:25] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-30 02:36:27] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-30 02:36:27] INFO:     127.0.0.1:36226 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-06-30 02:36:27] Child process unexpectedly failed with exitcode=9. pid=1994924
[2025-06-30 02:36:27] Child process unexpectedly failed with exitcode=9. pid=1994783


## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-30 02:36:38] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='XiaomiMiMo/MiMo-7B-RL', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=38069, mem_fraction_static=0.5, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1035806786, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, downl

[2025-06-30 02:36:50] Attention backend not set. Use flashinfer backend by default.
[2025-06-30 02:36:50] Init torch distributed begin.
[2025-06-30 02:36:50] Init torch distributed ends. mem usage=0.00 GB


[2025-06-30 02:36:51] Load weight begin. avail mem=78.50 GB


[2025-06-30 02:36:51] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.24it/s]


...

[2025-06-30 02:36:57] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=56.97 GB


[2025-06-30 02:36:58] INFO:     Started server process [1995688]
[2025-06-30 02:36:58] INFO:     Waiting for application startup.
[2025-06-30 02:36:58] INFO:     Application startup complete.
[2025-06-30 02:36:58] INFO:     Uvicorn running on http://0.0.0.0:38069 (Press CTRL+C to quit)


[2025-06-30 02:36:58] INFO:     127.0.0.1:44146 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-30 02:36:59] INFO:     127.0.0.1:44160 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-30 02:36:59] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-30 02:37:00] INFO:     127.0.0.1:44166 - "POST /generate HTTP/1.1" 200 OK
[2025-06-30 02:37:00] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-06-30 02:37:03] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-30 02:37:04] Decode batch. #running-req: 1, #token: 90, token usage: 0.00, accept len: 1.74, cuda graph: False, gen throughput (token/s): 11.10, #queue-req: 0


[2025-06-30 02:37:04] Decode batch. #running-req: 1, #token: 152, token usage: 0.01, accept len: 1.55, cuda graph: False, gen throughput (token/s): 98.29, #queue-req: 0


[2025-06-30 02:37:05] Decode batch. #running-req: 1, #token: 222, token usage: 0.01, accept len: 1.75, cuda graph: False, gen throughput (token/s): 108.78, #queue-req: 0


[2025-06-30 02:37:06] Decode batch. #running-req: 1, #token: 293, token usage: 0.01, accept len: 1.77, cuda graph: False, gen throughput (token/s): 108.05, #queue-req: 0


[2025-06-30 02:37:06] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, accept len: 1.73, cuda graph: False, gen throughput (token/s): 110.62, #queue-req: 0
[2025-06-30 02:37:06] INFO:     127.0.0.1:44170 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

[2025-06-30 02:37:06] Child process unexpectedly failed with exitcode=9. pid=1995971
[2025-06-30 02:37:06] Child process unexpectedly failed with exitcode=9. pid=1995837


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).